## Intro-notebook: GPT (OpenAI) og Claude (Anthropic) i Jupyter

To situasjoner: 1. **Jeg kjører lokalt** og 2. **Jeg kjører i skyen**

# Jeg kjører lokalt:

Den førtse del av noteboken hjelper deg å sette opp API-nøkler lokalt, teste at alt virker, og gjøre ditt første kall til både GPT-modeller fra OpenAI og Claude-modeller fra Anthropic.  
Målet er at du raskt kommer i gang, uten å dele nøklene dine med andre.

**Kort struktur:** først skaffer du egne nøkler, deretter lager du en `.env`-fil, så installerer du nødvendige Python-pakker, og til slutt kjører du to enkle testkall.


### 1) Skaff egne API-nøkler

Du trenger en nøkkel fra hver plattform. Du registrerer deg hos OpenAI og hos Anthropic. Når du er innlogget, finner du menyvalget for å generere en ny API-nøkkel i konsollen. Når nøklene er opprettet, kopierer du dem. De starter typisk med `sk-` for OpenAI og `sk-ant-` for Anthropic.

### 2) Lag en `.env`-fil lokalt (denne blir ignorert i `.gitignore`)

Gjør følgende i samme mappe som denne noteboken:

1. Opprett en ny tekstfil med navnet `.env`.
2. Lim inn to linjer, én for hver nøkkel, der du bytter ut plassholderne med dine faktiske nøkler:

```
OPENAI_API_KEY=sk-...din_nokkel_her...
ANTHROPIC_API_KEY=sk-ant-...din_nokkel_her...
```

3. Lagre filen. Ikke del den offentlig, og ikke legg den i et åpent Git-repo.

Hvis du foretrekker miljøvariabler i terminal i stedet for `.env`, kan du eksportere dem før du starter Jupyter:

```
export OPENAI_API_KEY="sk-..."
export ANTHROPIC_API_KEY="sk-ant-..."
```

### 3) Installer nødvendige pakker

Kjør cellen under for å installere tre pakker: `python-dotenv` for å lese `.env`, samt de offisielle klientbibliotekene for OpenAI og Anthropic.


In [30]:
# Installer nødvendige biblioteker (kjør én gang, eller når du bytter miljø)
# Dette er kun aktuelt om du kjører på Google Colab eller GitHub Codespaces:
# !pip install --quiet python-dotenv openai anthropic
# print("Pakker installert.")

# Kjører du lokalt (Anaconda med `ai-helse` conda environment)
# er python-dotenv, openai og  anthropic allerede installert


In [31]:
from dotenv import load_dotenv
import os, textwrap, sys

# Last .env hvis den finnes i samme mappe
_ = load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

problems = []
if not OPENAI_API_KEY or not OPENAI_API_KEY.strip():
    problems.append("Mangler OPENAI_API_KEY. Legg inn i .env eller som miljøvariabel.")
if not ANTHROPIC_API_KEY or not ANTHROPIC_API_KEY.strip():
    problems.append("Mangler ANTHROPIC_API_KEY. Legg inn i .env eller som miljøvariabel.")

if problems:
    msg = "\n".join(["❌ " + p for p in problems])
    raise RuntimeError(textwrap.dedent(f"""
    {msg}

    Tips:
      1) Opprett en .env-fil i samme mappe som noteboken.
      2) Sett OPENAI_API_KEY og ANTHROPIC_API_KEY.
      3) Kjør denne cellen på nytt.
    """))
else:
    print("✅ Fant begge API-nøkler i miljøet.")


✅ Fant begge API-nøkler i miljøet.



### 4) Første testkall: OpenAI (GPT)

Denne cellen bruker **OpenAI Python SDK** og `Responses`-endepunktet. Vi gjør en enkel tekstforespørsel for å bekrefte at tilkoblingen fungerer.


In [32]:

from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

prompt = "Skriv én setning som bekrefter at OpenAI-tilkoblingen fungerer (på norsk)."

# Responses API (anbefalt i ny SDK)
resp = client.responses.create(
    model="gpt-4o-mini",
    input=prompt,
)

# Ny SDK har en helper for å hente ut tekst
try:
    print("OpenAI svar:", resp.output_text)
except Exception:
    # Fallback i tilfelle struktur endres
    print(resp)


OpenAI svar: OpenAI-tilkoblingen fungerer som den skal.



### 5) Første testkall: Anthropic (Claude)

Denne cellen bruker **Anthropic Python SDK** og `Messages`-endepunktet. Vi sender en kort melding og skriver ut første tekstblokk i svaret.


In [33]:

from anthropic import Anthropic
import os

anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

user_msg = "Skriv én setning som bekrefter at Anthropic-tilkoblingen fungerer (på norsk)."

message = anthropic_client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    messages=[{"role": "user", "content": user_msg}],
)

# Skriv ut første tekstblokk hvis den finnes
text_blocks = [c.text for c in message.content if getattr(c, "type", "") == "text"]
print("Anthropic svar:", text_blocks[0] if text_blocks else message)


Anthropic svar: Anthropic-tilkoblingen fungerer perfekt, og jeg kan kommunisere med deg på norsk uten problemer.



### 6) Feilsøking og nyttige råd

Hvis du får en feil om manglende nøkler, sjekk at `.env` ligger i samme mappe og at du har kjørt cellen som laster den. Hvis OpenAI eller Anthropic gir en autorisasjonsfeil, kontroller at nøklene faktisk er aktive i konsollene deres, og at du har midler eller kreditter på kontoen. Hvis modellnavnene endrer seg, bytt til et støttet navn fra dokumentasjonen deres. For OpenAI finner du modeller og eksempler i API-dokumentasjonen. For Anthropic ligger modellnavn og parametre i seksjonen *Messages*.



# Jeg kjører i skyen:

Kortversjon: i skyen bør du bruke de innebygde “secrets”-løsningene i Colab og Codespaces, og så lese dem som miljøvariabler i notebooken. Ikke last opp `.env`-filer til repoet. Detaljene under dekker begge plattformer, med kopiér-lim-inn-klare snutter.

### Google Colab

Colab har en egen Secret-manager i UI-et. Du legger inn nøkler der én gang og henter dem programmatisk i notebooken. Dette er bedre enn å lime inn nøkler i celler eller `.env` i Google Drive. Colab eksponerer hemmeligheter via en liten modul:

In [34]:
# Check if running in Colab
try:
    from google.colab import userdata
    # This is Colab-specific code
    openai_key = userdata.get("OPENAI_API_KEY")
    anthropic_key = userdata.get("ANTHROPIC_API_KEY")

    # (valgfritt) speil til miljøvariabler for biblioteker som forventer os.getenv(...)
    os.environ["OPENAI_API_KEY"]   = openai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key

    # Sjekk
    if not openai_key or not anthropic_key:
        raise RuntimeError("Mangler Colab Secrets: legg inn OPENAI_API_KEY og ANTHROPIC_API_KEY i Colab → Secrets.")
except ImportError:
    print("Kjører ikke i Google Colab - sløyfer Colab-spesifikk setup")
    print("Hvis kjører lokalt, bruk løsningen med en .env fil som vist tidligere")

Kjører ikke i Google Colab - sløyfer Colab-spesifikk setup
Hvis kjører lokalt, bruk løsningen med en .env fil som vist tidligere


Hvor legger man inn? I Colab velger man **Secrets** i venstremenyen, oppretter `OPENAI_API_KEY` og `ANTHROPIC_API_KEY`, og gir notebooken tilgang. Dette grensesnittet er dokumentert og brukt i flere offisielle eksempler; koden over med `from google.colab import userdata` er mønsteret Colab selv viser i autentiserings-notebooks. ([Google Colab][[1](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb?utm_source=chatgpt.com)])

Praktiske tips i Colab:

* Secrets er knyttet til brukerkontoen, ikke notebooken. Du må selv legge inn dine nøkler. ([Medium][[2](https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0)])
* Runtime resetter miljøet. Secrets består, men du må kjøre cellen som henter dem hver økt. (Dette er forventet i Colab.) ([Google Colab][[1](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb?utm_source=chatgpt.com)])
* Håndter gjerne manglende hemmeligheter eksplisitt; `userdata.get()` gir en feil hvis nøkkelen ikke finnes. ([Stack Overflow][[3](https://stackoverflow.com/questions/77554491/catching-messageerror-exception-from-userdata-get-in-google-colaboratory-secre?utm_source=chatgpt.com)])

### GitHub Codespaces

Codespaces har “Development environment secrets” på tre nivåer: konto, organisasjon og repo. Når de er definert, dukker de opp som miljøvariabler i Codespace-terminalen og i Python via `os.getenv`. ([GitHub Docs][[4](https://docs.github.com/en/codespaces/managing-your-codespaces/managing-your-account-specific-secrets-for-github-codespaces?utm_source=chatgpt.com)])

Oppsett (student): Gå til **Settings → Secrets and variables → Codespaces** på egen GitHub-konto, legg til `OPENAI_API_KEY` og `ANTHROPIC_API_KEY`. I en organisasjon kan man også definere secrets for repo/orga. ([GitHub Docs][[4](https://docs.github.com/en/codespaces/managing-your-codespaces/managing-your-account-specific-secrets-for-github-codespaces?utm_source=chatgpt.com)])

Bruk i notebook/kode:

In [35]:
import os

openai_key = os.getenv("OPENAI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

if not openai_key or not anthropic_key:
    raise RuntimeError("Mangler Codespaces-secrets. Legg dem til i GitHub → Settings → Secrets and variables → Codespaces.")

Anbefalinger (vi vurderer å etablere) for vårt kursrepo:

* Legge inn “recommended secrets” i `devcontainer.json` slik at GitHub ber studentene fylle inn når de oppretter en Codespace. ([GitHub Docs][[5](https://docs.github.com/en/codespaces/setting-up-your-project-for-codespaces/configuring-dev-containers/specifying-recommended-secrets-for-a-repository?utm_source=chatgpt.com)])
* Dokumentér at secrets blir tilgjengelig som vanlige env-vars i shell og Python; dette er den støttede måten å bruke følsomme verdier i Codespaces. ([GitHub Docs][[6](https://docs.github.com/en/codespaces/developing-in-a-codespace/persisting-environment-variables-and-temporary-files?utm_source=chatgpt.com)])
* Vil vurdere å referere secrets i devcontainer-konfig (avansert), slik at de kan de leses via `localEnv`-mekanismer. ([Stack Overflow][[7](https://stackoverflow.com/questions/73052779/using-github-codespaces-secrets-in-devcontainer-json?utm_source=chatgpt.com)])

### Eksempelsnutter for første kall (begge plattformer)

OpenAI:

In [36]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
resp = client.responses.create(model="gpt-4o-mini", input="Skriv én setning som test.")
print(resp.output_text)

Selv de minste ting kan ha stor betydning.


Anthropic:

In [37]:
from anthropic import Anthropic
import os

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
msg = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=100,
    messages=[{"role": "user", "content": "Skriv én setning som test."}]
)
print(next((c.text for c in msg.content if getattr(c, "type", "")=="text"), msg))

Dette er en testsetning skrevet på norsk.


### Hva med `.env` i sky?

I Colab: ikke nødvendig — bruk Secrets-panelet og `userdata`. En opplastet `.env` i Google Drive er lett å lekke når notebook deles. I Codespaces: du *kan* ha en lokal `.env` i arbeidsmappen, men **ikke** committe den. Preferér Codespaces-secrets, som injiseres som env-vars for alle nye Codespaces-økter. ([GitHub Docs][[6](https://docs.github.com/en/codespaces/developing-in-a-codespace/persisting-environment-variables-and-temporary-files?utm_source=chatgpt.com)])

### Vanlige feil og tiltak

Hvis OpenAI/Anthropic sier “unauthorized” i Colab: sjekk at du hentet via `userdata.get(...)` og at notebooken har fått tilgang i Secrets-panelet. ([Google Colab][1])
Hvis env-vars er tomme i Codespaces: bekreft at du la secrets på riktig nivå (konto/orga/repo), og at du opprettet Codespace *etter* at secrets ble definert, eller at du har restartet terminal/VS Code vindu. ([GitHub Docs][4])

**Oppsummert:**

Bruk Colab **Secrets + `userdata.get`** i Google Colab; 
bruk **Codespaces Secrets** i GitHub Codespaces.

Les dem inn som miljøvariabler, gjør en eksplisitt nøkkel-sjekk i notebooken, og unngå å laste opp `.env` til repo eller dele nøkler i tekstceller. Dette gir studentene en trygg og forutsigbar flyt i begge skyløsninger. ([Google Colab][[1](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb?utm_source=chatgpt.com)])

```text
- [1]: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb?utm_source=chatgpt.com "Gemini API: Authentication Quickstart - Colab - Google"

- [2]: https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0?utm_source=chatgpt.com "How to Use Secrets in Google Colab for API Key Protection"

- [3]: https://stackoverflow.com/questions/77554491/catching-messageerror-exception-from-userdata-get-in-google-colaboratory-secre?utm_source=chatgpt.com "Catching MessageError exception from userdata.get() in ..."

- [4]: https://docs.github.com/en/codespaces/managing-your-codespaces/managing-your-account-specific-secrets-for-github-codespaces?utm_source=chatgpt.com "Managing your account-specific secrets for GitHub ..."

- [5]: https://docs.github.com/en/codespaces/setting-up-your-project-for-codespaces/configuring-dev-containers/specifying-recommended-secrets-for-a-repository?utm_source=chatgpt.com "Specifying recommended secrets for a repository"

- [6]: https://docs.github.com/en/codespaces/developing-in-a-codespace/persisting-environment-variables-and-temporary-files?utm_source=chatgpt.com "Persisting environment variables and temporary files"

- [7]: https://stackoverflow.com/questions/73052779/using-github-codespaces-secrets-in-devcontainer-json?utm_source=chatgpt.com "Using GitHub Codespaces secrets in devcontainer.json"
```